In [30]:
!pip install opencv-python opencv-python-headless

In [31]:
import subprocess
def run_shell_command(command):
    """Helper function to run shell commands and handle errors."""
    result = subprocess.run(command, capture_output=True, text=True, shell=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
    return result.stdout

In [32]:
# run this command to start the container
command = "docker run -it --rm --privileged -e DISPLAY=$DISPLAY -v /tmp/.X11-unix/:/tmp/.X11-unix:ro stella_vslam-iridescense"

In [33]:
# get the container id
import subprocess
print("Getting the first Docker container ID...")
container_id = subprocess.check_output("docker ps -q", shell=True).decode().strip()
print(f"Container ID: {container_id}")

Getting the first Docker container ID...
Container ID: 0745fd32b5e1


In [34]:
import yaml
yaml_content = """
# Equirectangular model for RICOH THETA S

#==============#
# Camera Model #
#==============#

Camera:
  name: "RICOH THETA S 960"
  setup: "monocular"
  model: "equirectangular"  # Ensure this matches your camera's model
  fps: 30.0
  cols: 1920
  rows: 960
  color_order: "RGB"
  k1: 0.0
  k2: 0.0
  k3: 0.0
  p1: 0.0
  p2: 0.0
  fx: 500.0                           # Focal length in the x-direction
  fy: 500.0                           # Focal length in the y-direction
  cx: 960.0                           # Principal point x-coordinate (center of the image)
  cy: 540.0
  baseline: 0.1   

#================#
# ORB Parameters #
#================#

Feature.max_num_keypoints: 2000
Feature.scale_factor: 1.2
Feature.num_levels: 8
Feature.ini_fast_threshold: 20
Feature.min_fast_threshold: 7
Feature.mask_rectangles:
  - [0.0, 1.0, 0.0, 0.1]
  - [0.0, 1.0, 0.84, 1.0]
  - [0.0, 0.2, 0.7, 1.0]
  - [0.8, 1.0, 0.7, 1.0]
"""

# Write the content to a file
file_path = './input/config.yaml'

with open(file_path, 'w') as file:
    file.write(yaml_content)

print(f"YAML file has been created at {file_path}")


YAML file has been created at ./input/config.yaml


In [35]:
# copy video and config to container
video_name = input("Please enter the video name: ")
print(f"Copying inputs to container {container_id}...")
subprocess.run(
    f"docker cp ./input {container_id}:/stella_vslam_examples/build/input",
    shell=True
)

Copying inputs to container 0745fd32b5e1...


CompletedProcess(args='docker cp ./input 0745fd32b5e1:/stella_vslam_examples/build/input', returncode=0)

In [36]:
# repair the container
subprocess.run(
    f"docker exec {container_id} mkdir output",
    shell=True
)
subprocess.run(
    f'docker exec {container_id} curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow',
    shell=True
)


CompletedProcess(args='docker exec 0745fd32b5e1 curl -sL "https://github.com/stella-cv/FBoW_orb_vocab/raw/main/orb_vocab.fbow" -o orb_vocab.fbow', returncode=0)

In [42]:
# run the container
subprocess.run(
    f"docker exec {container_id} ./run_video_slam \
    -v ./orb_vocab.fbow \
    -c ./input/config.yaml \
    -m ./input/{video_name} \
    --map-db-out map.msg \
    --no-sleep \
    --eval-log-dir output",
    shell=True
)


--start-timestamp is not set. using system timestamp.
If --no-sleep is set without --start-timestamp, timestamps may overlap between multiple runs.


[2025-01-03 07:10:04.806] [I] config file loaded: ./input/config.yaml
[2025-01-03 07:10:04.806] [I] 
original version of OpenVSLAM,
Copyright (C) 2019,
National Institute of Advanced Industrial Science and Technology (AIST)
All rights reserved.
stella_vslam (the changes after forking from OpenVSLAM),
Copyright (C) 2022, stella-cv, All rights reserved.

This is free software,
and you are welcome to redistribute it under certain conditions.
See the LICENSE file.

Camera:
  name: RICOH THETA S 960
  setup: monocular
  model: equirectangular
  fps: 30.0
  cols: 1920
  rows: 960
  color_order: RGB
  k1: 0.0
  k2: 0.0
  k3: 0.0
  p1: 0.0
  p2: 0.0
  fx: 500.0
  fy: 500.0
  cx: 960.0
  cy: 540.0
  baseline: 0.1
Feature.max_num_keypoints: 2000
Feature.scale_factor: 1.2
Feature.num_levels: 8
Feature.ini_fast_threshold: 20
Feature.min_fast_threshold: 7
Feature.mask_rectangles:
  - [0.0, 1.0, 0.0, 0.1]
  - [0.0, 1.0, 0.84, 1.0]
  - [0.0, 0.2, 0.7, 1.0]
  - [0.8, 1.0, 0.7, 1.0]

[2025-01-03 07:10:

CompletedProcess(args='docker exec 0745fd32b5e1 ./run_video_slam     -v ./orb_vocab.fbow     -c ./input/config.yaml     -m ./input/tokyo_0819_0829.mp4     --map-db-out map.msg     --no-sleep     --eval-log-dir output', returncode=0)

In [43]:
subprocess.run(
    f"docker cp {container_id}:/stella_vslam_examples/build/output .",
    shell=True
)

CompletedProcess(args='docker cp 0745fd32b5e1:/stella_vslam_examples/build/output .', returncode=0)

In [ ]:
# visualize animation
# run animation.py